In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import warnings
%matplotlib inline
warnings.filterwarnings('ignore')

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

np.random.seed(601)

import sklearn
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.preprocessing import LabelEncoder

# 2. 데이터 전처리
## Data Cleansing & Pre-Processing

In [2]:
data = pd.read_csv('data/201901-202003.csv')

In [3]:
data.head()
data.tail()

,REG_YYMM,CARD_SIDO_NM,CARD_CCG_NM,STD_CLSS_NM,HOM_SIDO_NM,HOM_CCG_NM,AGE,SEX_CTGO_CD,FLC,CSTMR_CNT,AMT,CNT
0,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,20s,1,1,4,311200,4
1,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,30s,1,2,7,1374500,8
2,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,30s,2,2,6,818700,6
3,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,40s,1,3,4,1717000,5
4,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,40s,1,4,3,1047300,3


,REG_YYMM,CARD_SIDO_NM,CARD_CCG_NM,STD_CLSS_NM,HOM_SIDO_NM,HOM_CCG_NM,AGE,SEX_CTGO_CD,FLC,CSTMR_CNT,AMT,CNT
24697787,202003,충북,충주시,휴양콘도 운영업,충북,충주시,30s,1,2,3,43300,4
24697788,202003,충북,충주시,휴양콘도 운영업,충북,충주시,40s,1,3,3,35000,3
24697789,202003,충북,충주시,휴양콘도 운영업,충북,충주시,50s,1,4,4,188000,6
24697790,202003,충북,충주시,휴양콘도 운영업,충북,충주시,50s,2,4,4,99000,6
24697791,202003,충북,충주시,휴양콘도 운영업,충북,충주시,60s,1,5,3,194000,3


In [4]:
data = data.fillna('')

In [5]:
# CARD_SIDO_NM HOM_SIDO_NM 비교해서 같은지 다른지로만 판단 

idx_0 = data[(data.CARD_SIDO_NM == data.HOM_SIDO_NM) == True].index
data.loc[idx_0, 'HOM_SIDO_NM'] = 0
idx_1 = data[(data.CARD_SIDO_NM == data.HOM_SIDO_NM) == False].index
data.loc[idx_1, 'HOM_SIDO_NM'] = 1
data

,REG_YYMM,CARD_SIDO_NM,CARD_CCG_NM,STD_CLSS_NM,HOM_SIDO_NM,HOM_CCG_NM,AGE,SEX_CTGO_CD,FLC,CSTMR_CNT,AMT,CNT
0,201901,강원,강릉시,건강보조식품 소매업,1,강릉시,20s,1,1,4,311200,4
1,201901,강원,강릉시,건강보조식품 소매업,1,강릉시,30s,1,2,7,1374500,8
2,201901,강원,강릉시,건강보조식품 소매업,1,강릉시,30s,2,2,6,818700,6
3,201901,강원,강릉시,건강보조식품 소매업,1,강릉시,40s,1,3,4,1717000,5
4,201901,강원,강릉시,건강보조식품 소매업,1,강릉시,40s,1,4,3,1047300,3
...,...,...,...,...,...,...,...,...,...,...,...,...
24697787,202003,충북,충주시,휴양콘도 운영업,1,충주시,30s,1,2,3,43300,4
24697788,202003,충북,충주시,휴양콘도 운영업,1,충주시,40s,1,3,3,35000,3
24697789,202003,충북,충주시,휴양콘도 운영업,1,충주시,50s,1,4,4,188000,6
24697790,202003,충북,충주시,휴양콘도 운영업,1,충주시,50s,2,4,4,99000,6


In [6]:
data.drop(['CARD_CCG_NM','HOM_CCG_NM','CSTMR_CNT'], inplace=True, axis=1)
data

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,HOM_SIDO_NM,AGE,SEX_CTGO_CD,FLC,AMT,CNT
0,201901,강원,건강보조식품 소매업,1,20s,1,1,311200,4
1,201901,강원,건강보조식품 소매업,1,30s,1,2,1374500,8
2,201901,강원,건강보조식품 소매업,1,30s,2,2,818700,6
3,201901,강원,건강보조식품 소매업,1,40s,1,3,1717000,5
4,201901,강원,건강보조식품 소매업,1,40s,1,4,1047300,3
...,...,...,...,...,...,...,...,...,...
24697787,202003,충북,휴양콘도 운영업,1,30s,1,2,43300,4
24697788,202003,충북,휴양콘도 운영업,1,40s,1,3,35000,3
24697789,202003,충북,휴양콘도 운영업,1,50s,1,4,188000,6
24697790,202003,충북,휴양콘도 운영업,1,50s,2,4,99000,6


In [7]:
merge = pd.read_csv('data/submission.csv', index_col=0)
merge.drop(['AMT'], axis=1, inplace=True)

index = merge[merge['REG_YYMM'] == 202007].index
merge.drop(index, axis=0, inplace=True)

merge.shape
merge

(697, 3)

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM
id,,,
0,202004,강원,건강보조식품 소매업
1,202004,강원,골프장 운영업
2,202004,강원,과실 및 채소 소매업
3,202004,강원,관광 민예품 및 선물용품 소매업
4,202004,강원,그외 기타 분류안된 오락관련 서비스업
...,...,...,...
692,202004,충북,피자 햄버거 샌드위치 및 유사 음식점업
693,202004,충북,한식 음식점업
694,202004,충북,호텔업


In [8]:
sub_col = []
for REG_YYMM in data['REG_YYMM'].unique():
    for CARD_SIDO_NM in merge['CARD_SIDO_NM'].unique():
        for STD_CLSS_CD in merge['STD_CLSS_NM'].unique():
            sub_col.append([REG_YYMM, CARD_SIDO_NM, STD_CLSS_CD])
sub_col = np.array(sub_col)
sub_col = pd.DataFrame(data=sub_col, columns=['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'])
sub_col['AMT'] = 1
sub_col

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
0,201901,강원,건강보조식품 소매업,1
1,201901,강원,골프장 운영업,1
2,201901,강원,과실 및 채소 소매업,1
3,201901,강원,관광 민예품 및 선물용품 소매업,1
4,201901,강원,그외 기타 분류안된 오락관련 서비스업,1
...,...,...,...,...
10450,202003,충북,피자 햄버거 샌드위치 및 유사 음식점업,1
10451,202003,충북,한식 음식점업,1
10452,202003,충북,호텔업,1
10453,202003,충북,화장품 및 방향제 소매업,1


In [9]:
df_dup = data[['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM']].drop_duplicates()
df_dup['AMT'] = 0
df_dup

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
0,201901,강원,건강보조식품 소매업,0
13,201901,강원,골프장 운영업,0
31,201901,강원,과실 및 채소 소매업,0
484,201901,강원,관광 민예품 및 선물용품 소매업,0
508,201901,강원,그외 기타 스포츠시설 운영업,0
...,...,...,...,...
24664590,202003,충북,기타 대형 종합 소매업,0
24664639,202003,충북,기타 수상오락 서비스업,0
24677055,202003,충북,택시 운송업,0
24677781,202003,충북,그외 기타 스포츠시설 운영업,0


In [10]:
df_dup['REG_YYMM'] = df_dup['REG_YYMM'].astype(str)
df_dup.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9438 entries, 0 to 24687181
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   REG_YYMM      9438 non-null   object
 1   CARD_SIDO_NM  9438 non-null   object
 2   STD_CLSS_NM   9438 non-null   object
 3   AMT           9438 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 368.7+ KB


In [11]:
df_merge = sub_col.merge(df_dup, left_on=['REG_YYMM','CARD_SIDO_NM', 'STD_CLSS_NM'], right_on=['REG_YYMM','CARD_SIDO_NM', 'STD_CLSS_NM'], how='outer')
df_merge

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT_x,AMT_y
0,201901,강원,건강보조식품 소매업,1,0.0
1,201901,강원,골프장 운영업,1,0.0
2,201901,강원,과실 및 채소 소매업,1,0.0
3,201901,강원,관광 민예품 및 선물용품 소매업,1,0.0
4,201901,강원,그외 기타 분류안된 오락관련 서비스업,1,NaN
...,...,...,...,...,...
10450,202003,충북,피자 햄버거 샌드위치 및 유사 음식점업,1,0.0
10451,202003,충북,한식 음식점업,1,0.0
10452,202003,충북,호텔업,1,0.0
10453,202003,충북,화장품 및 방향제 소매업,1,0.0


In [12]:
null_idx = df_merge[df_merge['AMT_y'].isnull() == False].index
null_idx

df_merge.drop(null_idx, axis=0, inplace=True)
df_merge

Int64Index([    0,     1,     2,     3,     5,     6,     7,     8,     9,
               10,
            ...
            10445, 10446, 10447, 10448, 10449, 10450, 10451, 10452, 10453,
            10454],
           dtype='int64', length=9438)

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT_x,AMT_y
4,201901,강원,그외 기타 분류안된 오락관련 서비스업,1,NaN
30,201901,강원,정기 항공 운송업,1,NaN
45,201901,경기,그외 기타 분류안된 오락관련 서비스업,1,NaN
53,201901,경기,내항 여객 운송업,1,NaN
55,201901,경기,면세점,1,NaN
...,...,...,...,...,...
10428,202003,충북,면세점,1,NaN
10429,202003,충북,버스 운송업,1,NaN
10437,202003,충북,여행사업,1,NaN
10442,202003,충북,자동차 임대업,1,NaN


In [ ]:
data.info()
data

In [13]:
df_merge['HOM_SIDO_NM'] = 0
df_merge['AGE'] = '00s'
df_merge['SEX_CTGO_CD'] = 0
df_merge['FLC'] = 0
df_merge['AMT'] = 0
df_merge['CNT'] = 0
df_merge.drop(['AMT_x','AMT_y'], axis=1, inplace=True)
df_merge

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,HOM_SIDO_NM,AGE,SEX_CTGO_CD,FLC,AMT,CNT
4,201901,강원,그외 기타 분류안된 오락관련 서비스업,0,00s,0,0,0,0
30,201901,강원,정기 항공 운송업,0,00s,0,0,0,0
45,201901,경기,그외 기타 분류안된 오락관련 서비스업,0,00s,0,0,0,0
53,201901,경기,내항 여객 운송업,0,00s,0,0,0,0
55,201901,경기,면세점,0,00s,0,0,0,0
...,...,...,...,...,...,...,...,...,...
10428,202003,충북,면세점,0,00s,0,0,0,0
10429,202003,충북,버스 운송업,0,00s,0,0,0,0
10437,202003,충북,여행사업,0,00s,0,0,0,0
10442,202003,충북,자동차 임대업,0,00s,0,0,0,0


In [14]:
concat_df = pd.concat([data, df_merge])
concat_df

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,HOM_SIDO_NM,AGE,SEX_CTGO_CD,FLC,AMT,CNT
0,201901,강원,건강보조식품 소매업,1,20s,1,1,311200,4
1,201901,강원,건강보조식품 소매업,1,30s,1,2,1374500,8
2,201901,강원,건강보조식품 소매업,1,30s,2,2,818700,6
3,201901,강원,건강보조식품 소매업,1,40s,1,3,1717000,5
4,201901,강원,건강보조식품 소매업,1,40s,1,4,1047300,3
...,...,...,...,...,...,...,...,...,...
10428,202003,충북,면세점,0,00s,0,0,0,0
10429,202003,충북,버스 운송업,0,00s,0,0,0,0
10437,202003,충북,여행사업,0,00s,0,0,0,0
10442,202003,충북,자동차 임대업,0,00s,0,0,0,0


In [15]:
data = concat_df
data

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,HOM_SIDO_NM,AGE,SEX_CTGO_CD,FLC,AMT,CNT
0,201901,강원,건강보조식품 소매업,1,20s,1,1,311200,4
1,201901,강원,건강보조식품 소매업,1,30s,1,2,1374500,8
2,201901,강원,건강보조식품 소매업,1,30s,2,2,818700,6
3,201901,강원,건강보조식품 소매업,1,40s,1,3,1717000,5
4,201901,강원,건강보조식품 소매업,1,40s,1,4,1047300,3
...,...,...,...,...,...,...,...,...,...
10428,202003,충북,면세점,0,00s,0,0,0,0
10429,202003,충북,버스 운송업,0,00s,0,0,0,0
10437,202003,충북,여행사업,0,00s,0,0,0,0
10442,202003,충북,자동차 임대업,0,00s,0,0,0,0


In [ ]:
data.STD_CLSS_NM.value_counts()

In [ ]:
data['STD_CLSS_CD'] = data['STD_CLSS_NM']
data['STD_CLSS_CD'].replace({'한식 음식점업':'한식'
                            ,'피자 햄버거 샌드위치 및 유사 음식점업':'한식'
                             ,'중식 음식점업':'한식'
                             ,'기타 외국식 음식점업':'한식'
                             ,'일식 음식점업':'한식'
                             ,'서양식 음식점업':'한식'
                             
                             , '체인화 편의점': '편의점'
                             , '슈퍼마켓': '편의점'
                             
                            ,'차량용 주유소 운영업': '운영업'
                            ,'골프장 운영업': '운영업'
                            ,'휴양콘도 운영업': '운영업'
                            ,'그외 기타 스포츠시설 운영업': '운영업'
                             ,'차량용 가스 충전업': '운영업'
                             
                             , '비알콜 음료점업':'음료점업'
                             
                             ,'기타 대형 종합 소매업':'소매업'
                            ,'빵 및 과자류 소매업':'소매업'
                            ,'기타음식료품위주종합소매업':'소매업'
                            ,'과실 및 채소 소매업':'소매업'
                            ,'화장품 및 방향제 소매업':'소매업'
                            ,'그외 기타 종합 소매업':'소매업'
                            ,'육류 소매업':'소매업'
                            ,'수산물 소매업':'소매업'
                            ,'건강보조식품 소매업':'소매업'
                            ,'관광 민예품 및 선물용품 소매업':'소매업'
                             ,'건강보조식품 소매업':'소매업'
                             
                            ,'스포츠 및 레크레이션 용품 임대업':'임대업'
                            ,'자동차 임대업':'임대업'
                             
                             ,'택시 운송업':'운송업'
                            ,'버스 운송업':'운송업'
                            ,'정기 항공 운송업':'운송업'
                            ,'내항 여객 운송업':'운송업'
                             
                             ,'기타 수상오락 서비스업':'서비스업'
                             ,'그외 기타 분류안된 오락관련 서비스업':'서비스업'
                             
                             ,'일반유흥 주점업':'주점업'
                             ,'기타 주점업':'주점업'
                             
                             , '면세점':'여행'
                             , '여행사업':'여행'
                             ,'호텔업':'여행'
                             ,'여관업':'여행'
                             
                            }, inplace=True)

In [16]:
data['AGE'] = data['AGE'].str.get(0)

### 날짜 처리 YYYYMM -> YYYY + MM

In [17]:
def grap_year(data):
    data = str(data)
    return int(data[:4])

def grap_month(data):
    data = str(data)
    return int(data[4:])

In [18]:
# all
data['year'] = data['REG_YYMM'].apply(lambda x: grap_year(x))
data['month'] = data['REG_YYMM'].apply(lambda x: grap_month(x))
#data = data.drop(['REG_YYMM'], axis=1)

In [ ]:
data

In [ ]:
def get_outlier(df=None, column=None, weight=1.5):
    # fraud에 해당하는 column 데이터만 추출, 1/4 분위와 3/4 분위 지점을 np.percentile로 구함. 
    
    fraud = df[column]
    quantile_25 = np.percentile(fraud.values, 25)
    quantile_75 = np.percentile(fraud.values, 75)
    
    # IQR을 구하고, IQR에 1.5를 곱하여 최대값과 최소값 지점 구함. 
    iqr = quantile_75 - quantile_25
    iqr_weight = iqr * weight
    lowest_val = quantile_25 - iqr_weight
    highest_val = quantile_75 + iqr_weight
    
    # 최대값 보다 크거나, 최소값 보다 작은 값을 아웃라이어로 설정하고 DataFrame index 반환. 
    #outlier_index = fraud[(fraud < lowest_val) | (fraud > highest_val)].index
    outlier_index = fraud[(fraud > highest_val)].index
    
    return outlier_index

In [ ]:
outlier_index = get_outlier(df=data, column='AMT', weight=1.5)
print('이상치 데이터 인덱스:', outlier_index)
data.drop(outlier_index, axis=0, inplace=True)

In [ ]:
data

### 데이터 정제

In [21]:
# all
df = data.copy()

columns = ['REG_YYMM','CARD_SIDO_NM', 'STD_CLSS_NM', 'HOM_SIDO_NM', 'AGE', 'SEX_CTGO_CD', 'FLC', 'year', 'month'] # , 'STD_CLSS_CD'
df = df.groupby(columns).sum().reset_index(drop=False)

In [ ]:
df

In [ ]:
df.info()

In [22]:
df['CARD_SIDO_NM']=df['CARD_SIDO_NM'].astype(str)
#df['STD_CLSS_CD']=df['STD_CLSS_CD'].astype(str)
df['STD_CLSS_NM']=df['STD_CLSS_NM'].astype(str)

df['REG_YYMM']=df['REG_YYMM'].astype(int)
df['HOM_SIDO_NM']=df['HOM_SIDO_NM'].astype(int)
df['AGE']=df['AGE'].astype(int)
df['SEX_CTGO_CD']=df['SEX_CTGO_CD'].astype(int)
df['FLC']=df['FLC'].astype(int)
df['year']=df['year'].astype(int)
df['month']=df['month'].astype(int)
df['AMT']=df['AMT'].astype(float)
df['CNT']=df['CNT'].astype(int)

### 인코딩

In [23]:
# all
dtypes = df.dtypes
encoders = {}
for column in df.columns:
    if str(dtypes[column]) == 'object':
        encoder = LabelEncoder()
        encoder.fit(df[column])
        encoders[column] = encoder
        
df_num = df.copy()        
for column in encoders.keys():
    encoder = encoders[column]
    df_num[column] = encoder.transform(df[column])
    
df_num    

LabelEncoder()

LabelEncoder()

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,HOM_SIDO_NM,AGE,SEX_CTGO_CD,FLC,year,month,AMT,CNT
0,201901,0,0,1,2,1,1,2019,1,311200.0,4
1,201901,0,0,1,3,1,2,2019,1,5169295.0,45
2,201901,0,0,1,3,2,2,2019,1,8618550.0,91
3,201901,0,0,1,4,1,2,2019,1,454420.0,3
4,201901,0,0,1,4,1,3,2019,1,6441165.0,60
...,...,...,...,...,...,...,...,...,...,...,...
198613,202003,16,14,0,0,0,0,2020,3,0.0,0
198614,202003,16,15,0,0,0,0,2020,3,0.0,0
198615,202003,16,23,0,0,0,0,2020,3,0.0,0
198616,202003,16,28,0,0,0,0,2020,3,0.0,0


In [ ]:
colormap = plt.cm.RdBu
plt.figure(figsize=(14, 12))
sns.heatmap(df_num.astype(float).corr(), linewidths=0.1, vmax=1.0,
           square=True, cmap=colormap, linecolor='white', annot=True, annot_kws={"size": 16})

# 3. 탐색적 자료분석
## Exploratory Data Analysis

In [ ]:
# 입력하세요.


# 4. 변수 선택 및 모델 구축
## Feature Engineering & Initial Modeling

In [ ]:
df_num.shape 
df_num 

In [ ]:
# 201908 ~ 202002 -> train
# 202003          -> test

train = df_num[df_num['REG_YYMM'] != 202003]
test = df_num[df_num['REG_YYMM'] == 202003]

In [ ]:
train.shape
test.shape

In [51]:
feature = df_num.drop([ 'REG_YYMM','AMT', 'CNT'], axis=1)
target = np.log1p(df_num['AMT'])

### feature, target 설정

In [ ]:
train_feature = train.drop([ 'REG_YYMM','AMT', 'CNT'], axis=1)
train_target = np.log1p(train['AMT'])

test_feature = test.drop([ 'REG_YYMM','AMT', 'CNT'], axis=1)
test_target = np.log1p(test['AMT'])

# 5. 모델 학습 및 검증
## Model Tuning & Evaluation

In [25]:
from __future__ import print_function # python 2에서 python 3 문법을 사용하려면

import torch

import torch.nn.init as init # 텐서 초기값 주기 위한 함수 

import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim # 경사하강 알고리즘 
from  torch.utils.data import DataLoader, TensorDataset 

import torchvision
import torchvision.transforms as transforms

from torch.autograd import Variable

In [26]:
torch.cuda.is_available()
torch.cuda.current_device()
torch.cuda.device_count()
torch.cuda.get_device_name(0)
torch.cuda.device(0)

True

0

1

'GeForce MX130'

In [27]:
torch.cuda.empty_cache()
cuda = torch.device('cuda')
cuda

device(type='cuda')

In [28]:
device = torch.device("cuda:0")
device

device(type='cuda', index=0)

In [ ]:
feature.shape, target.shape

In [ ]:
train_feature.shape, test_feature.shape, train_target.shape, test_target.shape

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(9, 256).cuda('cuda')
        self.fc2 = nn.Linear(256, 256).cuda('cuda')
        #self.fc3 = nn.Linear(256, 256).cuda('cuda')
        #self.fc4 = nn.Linear(256, 128).cuda('cuda')
        self.fc5 = nn.Linear(256, 128).cuda('cuda')
        self.fc6 = nn.Linear(128, 1).cuda('cuda')
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        #x = F.relu(self.fc3(x))
        #x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        x = F.dropout(x, training=self.training)
        x = F.relu(self.fc6(x))
        return x

model = Net()
model = model.cuda('cuda')

In [ ]:
#### 오차함수 객체
criterion = nn.MSELoss()
# 최적화 객체
optimizer = optim.Adam(model.parameters(), lr=0.1)

# 학습 시작 
for epoch in range(1000):
    total_loss = 0
    # 분할해 둔 데이터 꺼냄
    for train_x, train_y in train_loader:
        # 계산 그래프 구성
        train_x, train_y = Variable(train_x.float()), Variable(train_y.float())
        train_x = train_x.to('cuda')
        train_y = train_y.to('cuda')
        # 경사 초기화
        optimizer.zero_grad()
        # 순전파
        output = model(train_x).flatten()
        # 오차
        loss = criterion(output, train_y)
        # 역전파
        loss.backward()
        # 가중치 업데이터
        optimizer.step()
        # 누적 오차 계산 
        total_loss += loss.data
        
    if (epoch+1) % 10 == 0:
        print(epoch+1, total_loss)

In [117]:
feature = df_num.drop([ 'REG_YYMM','AMT', 'CNT'], axis=1)
target = np.log1p(df_num['AMT'])

In [118]:
# 훈련 데이터 텐서 변환
feature = torch.FloatTensor(np.asarray(feature))
target = torch.FloatTensor(np.asarray(target))

feature.shape, target.shape

feature.dtype

#feature = feature.reshape(-1,8,3)

#feature.shape


#train = TensorDataset(feature, target)
# 미니배치 분할
#train_loader = DataLoader(train, batch_size=32, shuffle=True) # , batch_size=100

# 미니배치 분할
#feature_loader = DataLoader(feature, shuffle=True) 
#target_loader = DataLoader(target, batch_size=32, shuffle=True) 

#feature = Variable(feature).to('cuda')
#target = Variable(target).to('cuda')

(torch.Size([198618, 8]), torch.Size([198618]))

torch.float32

In [122]:
model = nn.Sequential(
        nn.Linear(8,6),
        nn.ReLU(),
        nn.Linear(6,10),
        nn.ReLU(),
        nn.Linear(10,6),
        nn.ReLU(),
        nn.Linear(6,1),
    )

In [120]:
loss_func = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.1)

In [123]:
loss_arr =[]
label = Variable(target).to("cpu")

for i in range(10):
    optimizer.zero_grad()
    output = model(feature)
    output.to("cpu")
    loss = loss_func(output,label)
    loss.backward()
    optimizer.step()
    
    if i % 10 ==0:
        print(loss)
    
    loss_arr.append(loss.cpu().data.numpy())

tensor([[-40.9798],
        [-40.9996],
        [-41.0361],
        ...,
        [-41.7156],
        [-41.7657],
        [-41.7857]], grad_fn=<AddmmBackward>)

RuntimeError: $ Torch: not enough memory: you tried to allocate 146GB. Buy new RAM! at ..\aten\src\TH\THGeneral.cpp:201

In [113]:
torch.cuda.empty_cache()

In [77]:
feature.shape
feature.view()

torch.Size([66206, 8, 3])

In [65]:
class Net(nn.Module):
    
    def __init__(self):
        super(Net,self).__init__()
        self.layer = nn.Sequential(
            nn.Conv1d(8,16,3,1,1),
            nn.ReLU(),
            nn.Conv1d(16,32,5),
            nn.ReLU(),
            nn.MaxPool1d(2,2),
            nn.Conv2d(32,64,5),
            nn.ReLU(),
            nn.MaxPool1d(2,2)
        )
        self.fc_layer = nn.Sequential(
            nn.Linear(64*3*3,100),
            nn.ReLU(),
            nn.Linear(100,10)
        )       
        
    def forward(self,x):
        out = self.layer(x)
        out = out.view(batch_size,-1)
        out = self.fc_layer(out)

        return out
    
net = Net().cuda()#.double()    

In [32]:
criterion = nn.MSELoss()
optimizer = optim.SGD(net.parameters(), lr=0.1)

In [75]:
# 교차 엔트로피 손실과 모멘텀 값을 갖는 SGD

for epoch in range(1): # 데이터셋을 수차례 반복
    
    running_loss = 0.0
        
    for i, data in enumerate(feature_loader, 0):
    
        # [inputs, labels] 의 목록인 data로 부터 입력을 받은 후, 
        #inputs, labels = data
        
        print(data)
        #print('inputs',inputs)
        #print('labels',labels)
        
        data = Variable(data.float())
        data = data.to('cuda')
        
        #data = data.to(device='cuda')
        #labels = labels.to('cuda')
        
        #print(i, inputs.shape, labels.shape)        
        #inputs = inputs.reshape(900, 1, 1, 1)      
        #print(inputs.shape)        
        
        # 변화도(Gradient) 매개변수를 0으로 만들고
        optimizer.zero_grad()
        # 순전파 + 역전파 + 최적화를 한 후,
        
        #print('train_X', train_X.dtype,train_X.shape)
        outputs = net(data).flatten()        
        
        optimizer.zero_grad()
        
        feature, target = Variable(feature.float()), Variable(target.float())
        feature = feature.to('cuda')
        target = target.to('cuda')
        
        outputs = net(feature_loader).flatten()
        
        loss = criterion(outputs, target)
        
        loss.backward()
        optimizer.step()
        
        if i % 10 == 0:
            print(loss)        
        
print('Finished Training')            

tensor([[[5.0000e+00, 1.0000e+01, 1.0000e+00],
         [5.0000e+00, 1.0000e+00, 3.0000e+00],
         [2.0190e+03, 1.2000e+01, 5.0000e+00],
         [1.0000e+01, 1.0000e+00, 5.0000e+00],
         [1.0000e+00, 4.0000e+00, 2.0190e+03],
         [1.2000e+01, 5.0000e+00, 1.0000e+01],
         [1.0000e+00, 5.0000e+00, 2.0000e+00],
         [4.0000e+00, 2.0190e+03, 1.2000e+01]]], dtype=torch.float64)


RuntimeError: cuDNN error: CUDNN_STATUS_BAD_PARAM

In [49]:
66206/32

2068.9375

In [ ]:
# 훈련 데이터 텐서 변환
train_X = torch.from_numpy(np.asarray(train_feature).astype(float))
train_Y = torch.from_numpy(np.asarray(train_target).astype(float))

# 테스트 데이터 텐서 변환
test_X = torch.from_numpy(np.asarray(test_feature))
test_Y = torch.from_numpy(np.asarray(test_target).astype(float))

train_X.shape, train_Y.shape, test_X.shape, test_Y.shape

train_X = Variable(train_X).to('cuda')
train_Y = Variable(train_Y).to('cuda')

train = TensorDataset(train_X, train_Y)
# 미니배치 분할
train_loader = DataLoader(train, shuffle=True) # , batch_size=100

In [ ]:
1488912 - 186114

In [ ]:
class Net(nn.Module):
    
    def __init__(self):
        print('start init')
        super(Net, self).__init__()
                            # [6, 1, 5, 5],
        self.conv1 = nn.Conv2d(8 ,10, kernel_size=5).cuda('cuda')
        self.pool = nn.MaxPool2d(2,2).cuda('cuda')
        self.conv2 = nn.Conv2d(10,10, kernel_size=5).cuda('cuda')
        self.fc1 = nn.Linear(16*5*5, 120).cuda('cuda')
        self.fc2 = nn.Linear(120,84).cuda('cuda')
        self.fc3 = nn.Linear(84,1).cuda('cuda')
        print('end init')
        
    def forward(self, x):
        print('start forward')
        
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1,16*5*5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        print('end forward')
        return x
    
net = Net().cuda()#.double()    

In [ ]:
criterion = nn.MSELoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
net = nn.Sequential(
        nn.Linear(8, 8),
        nn.LeakyReLU(),
        nn.Linear(8,10),
        nn.ELU(),
        nn.Linear(10,6),
        nn.ReLU(),
        nn.Linear(6,1),
    ).cuda()

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(),lr=1)

In [ ]:
df_num.head(1)

In [ ]:
train_X[0]

In [ ]:
# 교차 엔트로피 손실과 모멘텀 값을 갖는 SGD

for epoch in range(1): # 데이터셋을 수차례 반복
    
    running_loss = 0.0

    for i in range(100):
        
        optimizer.zero_grad()
        
        train_X, train_Y = Variable(train_X.float()), Variable(train_Y.float())
        train_X = train_X.to('cuda')
        train_Y = train_Y.to('cuda')
        
        outputs = net(train_X).flatten()
        
        loss = criterion(outputs, train_Y)
        
        loss.backward()
        optimizer.step()
        
        if i % 10 == 0:
            print(loss)        

print('Finished Training')            

In [ ]:
# 교차 엔트로피 손실과 모멘텀 값을 갖는 SGD

for epoch in range(1): # 데이터셋을 수차례 반복
    
    running_loss = 0.0
    #for i, data in enumerate(train_loader, 0):
    for i in range(10):
        
        train_X, train_Y = Variable(train_X.float()), Variable(train_Y.float())
        train_X = train_X.to('cuda')
        train_Y = train_Y.to('cuda')
    
        # [inputs, labels] 의 목록인 data로 부터 입력을 받은 후, 
        #inputs, labels = data
        #inputs = inputs.to(device='cuda')
        #labels = labels.to('cuda')
        
        #print(i, inputs.shape, labels.shape)        
        #inputs = inputs.reshape(900, 1, 1, 1)      
        #print(inputs.shape)        
        
        # 변화도(Gradient) 매개변수를 0으로 만들고
        optimizer.zero_grad()
        # 순전파 + 역전파 + 최적화를 한 후,
        
        print('train_X', train_X.dtype,train_X.shape)
        outputs = net(train_X).flatten()
        
        print('outputs', outputs.shape)
        print('train_Y', train_Y.shape)
        
        loss = criterion(outputs, train_Y)
        
        print('loss', loss)
        
        loss.backward()
        
        optimizer.step()
        
        if i % 10 == 0:
            print(loss)        
        
        # 통계 출력
        """
        running_loss += loss.item()
        if i % 2000 == 1999: # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0
        """
print('Finished Training')            

# 미니 배치 없이 돌리는거 알아보기
# cnn 네트워크 만들기
# cnn 으로 돌리는거 성공해보자아!!!!!

In [ ]:
900/25

In [ ]:
torch.cuda.empty_cache()

In [ ]:
from sklearn.linear_model import LassoCV, Lasso

In [ ]:
model = Lasso(alpha=0.05)
model.fit(train_feature, train_target)
y_pred = model.predict(test_feature)

In [ ]:
model.fit(train_feature, train_target)

In [ ]:
y_pred = model.predict(test_feature)

In [ ]:
y_pred.shape

In [ ]:
# train test 

In [ ]:
# 훈련
model = ExtraTreesRegressor(n_jobs=-1, random_state=0)

In [ ]:
model.fit(train_feature, train_target)

In [ ]:
y_pred = model.predict(test_feature)

In [ ]:
def rmsle(y, pred):
    log_y = np.log1p(y)
    log_pred = np.log1p(pred)
    squared_error = (log_y - log_pred) ** 2
    rmsle = np.sqrt(np.mean(squared_error))
    return rmsle

In [ ]:
rmsle(test_target, y_pred)
# 0.22293679487049622  lasso
# 0.045193267838462725

In [ ]:
# all
model = ExtraTreesRegressor(n_jobs=-1, random_state=0)
model.fit(all_features, all_target)

In [ ]:
df_num

# 6. 결과 및 결언
## Conclusion & Discussion

In [ ]:
df_num

In [ ]:
# 예측 템플릿 만들기
REG_YYMMs     = [202004, 202007]
CARD_SIDO_NMs = df_num['CARD_SIDO_NM'].unique()
#STD_CLSS_CDs  = df_num['STD_CLSS_CD'].unique()
STD_CLSS_NMs  = df_num['STD_CLSS_NM'].unique()
HOM_SIDO_NMs  = df_num['HOM_SIDO_NM'].unique()
AGEs          = df_num['AGE'].unique()
SEX_CTGO_CDs  = df_num['SEX_CTGO_CD'].unique()
FLCs          = df_num['FLC'].unique()
years         = [2020]
months        = [4, 7]

In [ ]:
temp = []
for REG_YYMM in REG_YYMMs:
    for CARD_SIDO_NM in CARD_SIDO_NMs:
        for STD_CLSS_NM in STD_CLSS_NMs:
            for HOM_SIDO_NM in HOM_SIDO_NMs:
                for AGE in AGEs:
                    for SEX_CTGO_CD in SEX_CTGO_CDs:
                        for FLC in FLCs:
                            for year in years:
                                for month in months:
                                    temp.append([REG_YYMM, CARD_SIDO_NM, STD_CLSS_NM, HOM_SIDO_NM, AGE, SEX_CTGO_CD, FLC, year, month])
temp = np.array(temp)


In [ ]:
temp.shape
temp

In [ ]:
temp = torch.from_numpy(np.asarray(temp).astype(np.float32))
temp

In [ ]:
result = torch.max(model(temp).data, 1)[1]

In [ ]:
result.shape
result

In [ ]:
pred = np.expm1(result)
pred
np.round(pred, 0)

In [ ]:
temp['AMT'] = np.round(pred, 0)

In [ ]:
model = Lasso(alpha=0.05)
model.fit(feature, target)

In [ ]:
# 예측
pred = model.predict(temp)
pred = np.expm1(pred)
pred.shape
pred

In [ ]:
temp['AMT'] = np.round(pred, 0)
#temp['REG_YYMM'] = temp['year']*100 + temp['month']
temp = temp[['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM', 'AMT']]
temp = temp.groupby(['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM']).sum().reset_index(drop=False)

In [ ]:
# 디코딩 
temp['CARD_SIDO_NM'] = encoders['CARD_SIDO_NM'].inverse_transform(temp['CARD_SIDO_NM'])
temp['STD_CLSS_NM'] = encoders['STD_CLSS_NM'].inverse_transform(temp['STD_CLSS_NM'])

In [ ]:
temp

In [ ]:
# 제출 파일 만들기
submission = pd.read_csv('data/submission.csv', index_col=0)
submission = submission.drop(['AMT'], axis=1)
submission = submission.merge(temp, left_on=['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'], right_on=['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'], how='left')
submission.index.name = 'id'
submission.to_csv('20200712_2.csv', encoding='utf-8-sig')
submission.head(30)